In [ ]:
%load_ext autoreload
%autoreload 2

<h3> Загрузка библиотек

In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

In [2]:
from scripts.load_and_save import (get_dcm_info, get_dcm_vol, vox_size2affine,
                                   save_vol_as_nii, load_sample_data)
from scripts.load_and_save import load_nii_vol, save_vol_as_nii, load_sample_data

from ml.models.unet3d import U_Net
from ml.models.rog import ROG
from ml.models.unet_deepsup import U_Net_DeepSup

from ml.utils import get_total_params, save_model, load_pretrainned
from ml.dataset import preprocess_dataset, HVB_Dataset, norm_vol
from ml.trainer import Trainer
from ml.losses import ExponentialLogarithmicLoss, WeightedExpBCE, TverskyLoss, IOU_Metric, MultyscaleLoss

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


<h3> Создание экзепляра класса датасет

In [4]:
train_dataset_settings = {
    "data_dir" : "/home/msst/Documents/medtech/brain_seg_dataset",
    "patch_shape" : (64, 64, 64),
    "number_of_patches" : 256,
    "mode": "train",
    "RAM_samples" : True 
}
patch_data_df, sample_data_df = preprocess_dataset(train_dataset_settings)
train_dataset = HVB_Dataset(train_dataset_settings)

/home/msst/Documents/medtech/brain_seg_dataset/P12_CTA(no_brain) P12_CTA(no_brain)
/home/msst/Documents/medtech/brain_seg_dataset/CT_S5020_uint16 CT_S5020_uint16
/home/msst/Documents/medtech/brain_seg_dataset/P62_CTA(no_brain) P62_CTA(no_brain)


In [5]:
val_dataset_settings = {
    "data_dir" : "/home/msst/Documents/medtech/brain_seg_dataset",
    "patch_shape" : (256, 256, 128),
    "mode": "eval",
    "RAM_samples" : train_dataset_settings["RAM_samples"] 
}
val_dataset = HVB_Dataset(val_dataset_settings)

In [22]:
test_dataset_settings = {
    "data_dir" : "/home/msst/Documents/medtech/brain_seg_dataset_test",
    "patch_shape" : (256, 256, 128),
    "mode": "eval",
    "number_of_patches" : 0,
    "RAM_samples" : False
}
preprocess_dataset(test_dataset_settings)
test_dataset = HVB_Dataset(test_dataset_settings)

/home/msst/Documents/medtech/brain_seg_dataset_test/P35_CTA 0.625mm(no_seg) P35_CTA 0.625mm(no_seg)
/home/msst/Documents/medtech/brain_seg_dataset_test/P70_CTA 0.625mm(no_seg) P70_CTA 0.625mm(no_seg)
/home/msst/Documents/medtech/brain_seg_dataset_test/P28_CTA_(0.42, 0.42, 0.3)(no_seg) P28_CTA_(0.42, 0.42, 0.3)(no_seg)


In [23]:
train_loader_params = {"batch_size": 16,
                 "shuffle": True,
                 "num_workers": 6
                }

val_loader_params = {"batch_size": 1,
                     "shuffle": False,
                     "num_workers": 6
                    }

train_dataloader = DataLoader(train_dataset, **train_loader_params)
val_dataloader = DataLoader(val_dataset, **val_loader_params)
test_dataloader = DataLoader(test_dataset, **val_loader_params)

In [24]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

768
3
3


<h3> Создание экземпляра модели

In [9]:
#rog_params = {
#    'classes': 1,
#    'modalities': 1,
#    'strides': [[2, 2, 1], [2, 2, 1], [2, 2, 2]],
#}
#model = ROG(rog_params)

In [10]:
#model = U_Net()
class swish(nn.Module):
    def forward(self, input_tensor):
        return input_tensor * torch.sigmoid(input_tensor)

#act_fn: nn.PReLU(inplace=True), nn.ReLU(inplace=True), swish
model = U_Net_DeepSup(channel_coef=16, act_fn=swish())

In [11]:
print('Number of parameters: {}'.format(get_total_params(model)))

Number of parameters: 6477875


In [12]:
import lightning as L


class LitModel(L.LightningModule):
    def __init__(self, torch_model, loss_fn, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate
        self.model = torch_model
        self.loss_fn = loss_fn

    def training_step(self, batch, batch_idx):
        head_batch = batch['head_patch'].to(self.device)
        vessels_batch = batch['vessels_patch'].to(self.device)
    
        outputs = self.model.forward(head_batch)
        loss = self.loss_fn(vessels_batch, outputs)
        self.log("train_loss", loss)
        return(loss)
    
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

#loss_fn = MultyscaleLoss(ExponentialLogarithmicLoss(gamma_tversky = 0.5, gamma_bce = 0.5, lamb=0.01,
#                                                    freq = 0.001, tversky_alfa=0.5))
#lightning_model = LitModel(model, loss_fn, 0.01)
#trainer = L.Trainer(accelerator='gpu', auto_lr_find=True)
#trainer.tune(lightning_model, train_dataloaders=train_dataloader)
#print("predicted best lr:", lightning_model.learning_rate)

<h3> Обучение модели

In [13]:
#loss_fn = nn.BCELoss(reduction='mean')

#loss_fn = ExponentialLogarithmicLoss(gamma_tversky = 1, gamma_bce = 1, lamb=1, freq = 0.001)
#loss_fn.weighted_bce_loss.bce_weight = 1

#print(loss_fn.weighted_bce_loss.bce_weight)

#loss_fn = WeightedExpBCE(0.5)
#loss_fn = TverskyLoss(0.5)

In [14]:
#l = WeightedExpBCE(0.5)
#l.set_bce_weight(0.001)
#loss_fn = MultyscaleLoss(l)

#loss_fn = MultyscaleLoss(TverskyLoss(0.75))

loss_fn = MultyscaleLoss(ExponentialLogarithmicLoss(gamma_tversky = 1, gamma_bce = 1, lamb=0.9,
                                                    freq = 0.001, tversky_alfa=0.75))

In [15]:
metric_fn = IOU_Metric()

trainer_config = {
    'n_epochs': 100,
    "loss" : loss_fn,
    "metric" : metric_fn,
    'device' : device,
    "optimizer_fn" : lambda model: torch.optim.ASGD(model.parameters(), lr=0.25),
    "sheduler_fn": lambda optimizer: StepLR(optimizer, step_size=5, gamma=0.5) 
}
trainer = Trainer(trainer_config)

In [16]:
model = trainer.fit(model, train_dataloader, val_dataloader)

Epoch 1/100


  0%|                                                    | 0/48 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [17]:
model_name = "UnetMSS16_logTversky_100"
#trainer.save("/home/msst/repo/MSRepo/VesselSegmentation/saved_models/" + model_name)

In [18]:
model.load_state_dict(torch.load("/home/msst/repo/MSRepo/VesselSegmentation/saved_models/" + model_name)["model_state_dict"])
trainer.model = model.to(device)

<h3> Сегментация с помощью обученной модели
    

In [25]:
#patch_shape = (256, 256, 128)
data_loader = test_dataloader#val_dataloader

patch_shape = data_loader.dataset.patch_shape
for batch in data_loader:
    head_batch = batch['head']
    vessels_batch = batch['vessels']
    affine = batch['affine'][0]
    sample_name = batch['sample_name'][0]
    print("###", sample_name, "###")
    
    head_seg = trainer.predict(head_batch, patch_shape)
    metric = trainer.metric_fn(head_seg, vessels_batch)
    
    print('metric:', metric, "sum/sum_GT:", head_seg.sum()/vessels_batch.sum())
    
    vessels_seg = head_seg[0, 0]
    data_dir = "seg_data/" + model_name
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)

    path_to_save_vessels = data_dir + '/' + sample_name + '.nii.gz'
    save_vol_as_nii(vessels_seg, affine, path_to_save_vessels)

1 False1
 False
1 False
/home/msst/Documents/medtech/brain_seg_dataset_test/P70_CTA 0.625mm(no_seg) P70_CTA 0.625mm(no_seg)
2
/home/msst/Documents/medtech/brain_seg_dataset_test/P35_CTA 0.625mm(no_seg) P35_CTA 0.625mm(no_seg)
2
### P35_CTA 0.625mm(no_seg) ###
/home/msst/Documents/medtech/brain_seg_dataset_test/P28_CTA_(0.42, 0.42, 0.3)(no_seg) P28_CTA_(0.42, 0.42, 0.3)(no_seg)
2
metric: tensor([8.0569e-05]) sum/sum_GT: tensor(0.0005)
### P70_CTA 0.625mm(no_seg) ###
metric: tensor([4.5899e-05]) sum/sum_GT: tensor(0.0002)
### P28_CTA_(0.42, 0.42, 0.3)(no_seg) ###
metric: tensor([0.0001]) sum/sum_GT: tensor(2.2750e-05)


In [ ]:
print(head_seg.sum())
vessels_seg = head_seg[0, 0]
print(vessels_seg.sum())

In [31]:
data_dir = "seg_data/P12_CTA"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)


path_to_save_vessels = data_dir + '/' + model_name + '_2.nii.gz'
save_vol_as_nii(vessels_seg, affine, path_to_save_vessels)

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
